In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchainhub

In [2]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Agents

Ok, this is awesome, we can now use openai functions connected to the ChatGPT API endpoint to allow it to perform actions in the real world, and the calling of said functions is made easier by leveraging the json schema which structures how the model would call the function. 

This is a great setup and tool but more complex tasks, we want to have more control over the process these potential agents will go through in order to make them as reliable as possible. We want to control things like:

- What goes in and out of prompts
- What goes in and out of each thought and action pair stage the agent goes through when doing something in the real-world
- A convenient interface to compose agents with useful building blocks (also leverage open source LLMs if we want to)

I could go on but this is enough for now.

For scenarios like these where just connecting a model to some tools won't cut it, a really great framework that has had a lot of sucess recently is [LangChain](https://python.langchain.com/docs/get_started/introduction).

LangChain is a framework that allows for building LLM-powered applications by giging developers the ability to build and compose building blocks like prompts, models, tools and so on to develop complex and interesting applications.

Here are the steps for building custom agents with Langchain:

1. Setup the LLM
2. Define the tool or list of tools
3. Set up a prompt template
4. Bind the llm with the tools
5. Define the agent as a dictionary with keys: input, and agent scratchpad
6. Use LCEL to connect agent, prompt and the LLM binded with tools
7. Create the agent loop
8. Wrap everything under `AgentExecutor`
9. Invoke the agent with some query input

Let's take a look below at a LangChain implementation of our simple agent that can create directories:

In [16]:
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.tools import tool
from langchain.agents import AgentExecutor
import subprocess

@tool
def create_directory(directory_name):
    """Function that creates a directory with the given name."""
    subprocess.run(["mkdir", directory_name])

In [17]:
prompt = hub.pull("hwchase17/openai-tools-agent")

In [23]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [24]:
llm = ChatOpenAI(model="gpt-4o")

In [25]:
tools = [create_directory]

In [26]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [27]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages

In [28]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c1867281

In [30]:
agent_executor.invoke({"input": "Create a folder called 'lucas-agent-007-the-return-of-pancake-man'."})



> Entering new AgentExecutor chain...

Invoking: `create_directory` with `{'directory_name': 'lucas-agent-007-the-return-of-pancake-man'}`


None

mkdir: lucas-agent-007-the-return-of-pancake-man: File exists


The folder 'lucas-agent-007-the-return-of-pancake-man' has been created successfully.

> Finished chain.


{'input': "Create a folder called 'lucas-agent-007-the-return-of-pancake-man'.",
 'output': "The folder 'lucas-agent-007-the-return-of-pancake-man' has been created successfully."}

In [12]:
!ls -d */

agent-deploy/
assets-resources/
lucas-agent-007-the-return-of-pancake-man/
lucas-agent-007/
lucas-loves-pancakes/
lucas-still-loves-pancakes/
lucas-test/
lucas-the-agent-master/
pancakes-are-better-than-waffles/
pancakes-rule/
test/


In [31]:
import subprocess
from langchain.tools import tool
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
import json
import os

@tool
def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    os.makedirs(directory_name, exist_ok=True)
    return json.dumps({"directory_name": directory_name})
    

tools = [create_directory]

llm_chat = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind_tools(tools)

agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_tool_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIToolsAgentOutputParser())

In [32]:
agent

{
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are very powerful assistant that helps                users perform tasks in the terminal.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x136a8a590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12

In [33]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable={
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are very powerful assistant that helps                users perform tasks in the terminal.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x136a8a590>, async_clien

In [35]:
action_input = "Create a folder called 'lucas-still-loves-pancakes'."

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `create_directory` with `{'directory_name': 'lucas-still-loves-pancakes'}`


{"directory_name": "lucas-still-loves-pancakes"}The directory 'lucas-still-loves-pancakes' has been created successfully.

> Finished chain.


{'input': "Create a folder called 'lucas-still-loves-pancakes'.",
 'output': "The directory 'lucas-still-loves-pancakes' has been created successfully."}

When we use an agent we want that agent to have access to its previous outputs and decision-making process in order for it to make more informed decisions. Therefore, LangChain allows you to set all that up yourself so you have ultimate control over what is going on.

On the next notebook we'll dive into the basics of LangChain for building Agents. 

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)